In [1169]:
import pandas as pd
import numpy as np
import kagglehub
from collections import defaultdict
import random
from scipy.stats import t


In [513]:
#cleaned up and formated data from kenpom
df = pd.read_csv("marchmadness/formatted_kenpom.csv")  


In [ ]:

random.seed(142)  # For reproducibility

teams_list = df["Team"].tolist() 

tournament_df = df[df["Team"].isin(teams_list)].drop_duplicates(subset=["Team"]).copy()

#corrections based on thougher schedules and opponents
tournament_df["Adj_NetRtg"] = tournament_df["NetRtg"] + 0.1 * (tournament_df["SS_NetRtg"] - tournament_df["SS_NetRtg"].mean())
conf_strength = tournament_df.groupby("Conf")["NetRtg"].mean().to_dict()
tournament_df["Conf_Adjustment"] = tournament_df["Adj_NetRtg"] + 0.1 * (tournament_df["Conf"].map(conf_strength) - tournament_df["Adj_NetRtg"].mean())

#some normalizations
max_abs_netrtg = max(abs(tournament_df["Conf_Adjustment"].min()), tournament_df["Conf_Adjustment"].max())
tournament_df["Conf_Adjustment"] = tournament_df["Conf_Adjustment"] / max_abs_netrtg

# Compute the league average and standard deviation of NetRtg.
league_avg = tournament_df["Conf_Adjustment"].mean()
league_std = tournament_df["Conf_Adjustment"].std()

#for sigma or volatility, use the W-L ratio, if teams either win a lot or lose a lot we expect them to keep doing that
# if they do both in equal portions this means its harder to predict

baseline_sigma = tournament_df["Conf_Adjustment"].std()

def parse_win_loss(record):
    wins, losses = record.split('-')
    return int(wins), int(losses)

tournament_df["Wins"] = tournament_df["W-L"].apply(lambda x: parse_win_loss(x)[0])
tournament_df["Losses"] = tournament_df["W-L"].apply(lambda x: parse_win_loss(x)[1])
tournament_df["WinPct"] = tournament_df["Wins"] / (tournament_df["Wins"] + tournament_df["Losses"])

baseline_sigma = 0.3

tournament_df["Sigma_WL"] = baseline_sigma * (tournament_df["WinPct"] * (1 - tournament_df["WinPct"])) / 0.25

team_sigma = tournament_df.set_index("Team")["Sigma_WL"].to_dict()
teams = tournament_df.set_index("Team")["Conf_Adjustment"].to_dict()

#use a zscore on luck factor, this is a factor that checks if teams overperform typically compared to their expected win chances
luck_mean = tournament_df["Luck"].mean()
luck_std = tournament_df["Luck"].std()
team_luck = tournament_df.set_index("Team")["Luck"].apply(lambda x: (x - luck_mean) / luck_std).to_dict()

# Define team_momentum as the z-score of NetRtg (as a proxy for momentum)
#the logic here is, if they overperform compared to the average of their league they should feel great
league_avg_netrtg = tournament_df["NetRtg"].mean()
league_std_netrtg = tournament_df["NetRtg"].std()
team_momentum = tournament_df.set_index("Team")["NetRtg"].apply(lambda x: (x - league_avg_netrtg) / league_std_netrtg).to_dict()

real_seeds = {
    # South Region
    "Auburn": (1, "South"), "Alabama St.": (16, "South"), #"Saint Francis": (16, "South"), FIRST FOUR
    "Louisville": (8, "South"), "Creighton": (9, "South"),
    "Michigan": (5, "South"), "UC San Diego": (12, "South"),
    "Texas A&M": (4, "South"), "Yale": (13, "South"),
    "Mississippi": (6, "South"), "San Diego St.": (11, "South"), #"North Carolina": (11, "South"), FIRST 4
    "Iowa St.": (3, "South"), "Lipscomb": (14, "South"),
    "Marquette": (7, "South"), "New Mexico": (10, "South"),
    "Michigan St.": (2, "South"), "Bryant": (15, "South"),

        # East Region
    "Duke": (1, "East"), "American": (16, "East"), # "Mount St. Mary's": (16, "East"), DEPENDS ON FIRST 4
    "Mississippi St.": (8, "East"), "Baylor": (9, "East"),
    "Oregon": (5, "East"), "Liberty": (12, "East"),
    "Arizona": (4, "East"), "Akron": (13, "East"),
    "BYU": (6, "East"), "VCU": (11, "East"),
    "Wisconsin": (3, "East"), "Montana": (14, "East"),
    "Saint Mary's": (7, "East"), "Vanderbilt": (10, "East"),
    "Alabama": (2, "East"), "Robert Morris": (15, "East"),

        # Midwest Region
    "Houston": (1, "Midwest"), "SIUE": (16, "Midwest"),
    "Gonzaga": (8, "Midwest"), "Georgia": (9, "Midwest"),
    "Clemson": (5, "Midwest"), "McNeese": (12, "Midwest"),
    "Purdue": (4, "Midwest"), "High Point": (13, "Midwest"),
    "Illinois": (6, "Midwest"), "Texas": (11, "Midwest"), #, "Xavier": (11, "Midwest"), FIRST FOUR
    "Kentucky": (3, "Midwest"), "Troy": (14, "Midwest"),
    "UCLA": (7, "Midwest"), "Utah St.": (10, "Midwest"),
    "Tennessee": (2, "Midwest"), "Wofford": (15, "Midwest"),

        # West Region
    "Florida": (1, "West"), "Norfolk St.": (16, "West"),
    "Connecticut": (8, "West"), "Oklahoma": (9, "West"),
    "Memphis": (5, "West"), "Colorado St.": (12, "West"),
    "Maryland": (4, "West"), "Grand Canyon": (13, "West"),
    "Missouri": (6, "West"), "Drake": (11, "West"),
    "Texas Tech": (3, "West"), "UNC Wilmington": (14, "West"),
    "Kansas": (7, "West"), "Arkansas": (10, "West"),
    "St. John's": (2, "West"), "Nebraska Omaha": (15, "West")
}

tournament_df["Seed"] = tournament_df["Team"].map(lambda x: real_seeds.get(x, (None, None))[0])
tournament_df["Region"] = tournament_df["Team"].map(lambda x: real_seeds.get(x, (None, None))[1])

main_bracket_teams = tournament_df[tournament_df["Team"].isin(real_seeds.keys())].copy()

main_bracket_teams["Seed"] = main_bracket_teams["Seed"].astype(int)

main_bracket_teams = main_bracket_teams.sort_values(by=["Region", "Seed"]).reset_index(drop=True)

first_round_matchups = generate_first_round_matchups(main_bracket_teams)  # returns a list of 32 matchups

print("Tournament Start:")
print("-----------------------------------------------------------")
# Print first round matchups 
print("\nRound of 64:")
for matchup in first_round_matchups:
    print(f"{matchup[0]} vs {matchup[1]}")


In [1234]:

def simulate_full_bracket(first_round_matchups, n_sim=10000):

    full_bracket_paths = defaultdict(int)
    bar_length = 50 

    for sim in range(n_sim):
        team_fatigue = defaultdict(lambda: 1.0)

        current_matchups = first_round_matchups[:]  

        bracket_path = {}
        round_number = 1  
        
        if sim % 100 == 0 or sim == n_sim - 1:
            progress = sim / n_sim
            filled_length = int(round(bar_length * progress))
            bar = '#' * filled_length + '-' * (bar_length - filled_length)
            print(f"Simulation Progress: [{bar}] {progress * 100:.2f}% completed", end='\r')
            
        round_label = "Round of 64"

        round_results = []
        for matchup in current_matchups:
            outcome, p = simulate_game(matchup, round_number, team_fatigue)
            round_results.append(outcome[0])
            closeness = 1 - 2 * abs(p - 0.5) 
            team_fatigue[outcome[0]] *= (1 + params["fatigue_param"] * closeness)

        bracket_path[round_label] = tuple(round_results)
        
        while len(current_matchups) > 0:
            round_number += 1
            winners = []
            round_label = f"Round {round_number}"

            for matchup in current_matchups:
                outcome, p = simulate_game(matchup, round_number, team_fatigue)
                winners.append(outcome[0])

                closeness = 1 - 2 * abs(p - 0.5)
                team_fatigue[outcome[0]] *= (1 + params["fatigue_param"] * closeness)

            bracket_path[round_label] = tuple(winners)

            if len(winners) == 1:
                break

            current_matchups = [(winners[i], winners[i+1]) for i in range(0, len(winners), 2)]

        champion = winners[0]
        bracket_path["Champion"] = champion
        
        key = tuple((k, bracket_path[k]) for k in sorted(bracket_path.keys()))
        full_bracket_paths[key] += 1
        
    print(f"Simulation Progress: [{'#' * bar_length}] 100.00% completed")
    return full_bracket_paths

def count_champions(full_bracket_paths):

    champion_counts = defaultdict(int)
    for path, count in full_bracket_paths.items():

        for round_label, outcome in path:
            if round_label == "Champion":
                champion_counts[outcome] += count
    return champion_counts

def print_champion_counts(champion_counts, n_sim, top_n=8):

    print(f"\nChampion Win Counts (Top {top_n}): ")

    sorted_champs = sorted(champion_counts.items(), key=lambda x: x[1], reverse=True)[:top_n]
    for champion, count in sorted_champs:
        percentage = count / n_sim * 100
        print(f"{champion}: {count} wins ({percentage:.2f}%)")

In [1235]:
#functions to simulate the rounds multiple times before moving on to the next round

BOLD = "\033[1m"
RESET = "\033[0m"

games = {1:0,2:32, 3:46, 4:54, 5: 58, 6: 60}

def simulate_round_by_mode(matchups, n_sim=1000, round_number=1, team_fatigue=None):
    bar_length = 50 
    if team_fatigue is None:
        team_fatigue = defaultdict(lambda: 1.0)
    winners = []
    details = [] 
    ngames = 0
    for matchup in matchups:
        ngames+=1
        number = (ngames + games[round_number])*n_sim

        if number % 100 == 0 or number == n_sim - 1:
            progress = number / (60*n_sim)
            filled_length = int(round(bar_length * progress))
            bar = '#' * filled_length + '-' * (bar_length - filled_length)
            print(f"Simulation Progress: [{bar}] {progress * 100:.2f}% completed", end='\r')
           
        win_probs = []
        win_counts = {}
        for _ in range(n_sim):
            outcome, p = simulate_game(matchup, round_number=round_number, team_fatigue=team_fatigue)
            win_probs.append(p)
            winner = outcome[0]
            win_counts[winner] = win_counts.get(winner, 0) + 1

        best_team, best_count = max(win_counts.items(), key=lambda x: x[1])

        if len(matchup) == 3:
            team_a, team_b, _ = matchup
        else:
            team_a, team_b = matchup
        avg_p = np.mean(win_probs)

        if best_team == team_b:
            avg_p = 1 - avg_p

        closeness = 1 - 2 * abs(avg_p - 0.5)  # 1 if exactly 0.5, 0 if extreme.

        team_fatigue[best_team] *= (1+fatigue_param * closeness)
        winners.append(best_team)
        details.append((team_a, team_b, best_team, best_count, avg_p, closeness, team_fatigue[best_team]))

    if len(winners) % 2 == 0:
        next_matchups = [(winners[i], winners[i+1]) for i in range(0, len(winners), 2)]
    else:
        next_matchups = []
    return details, winners, next_matchups, team_fatigue

def simulate_tournament_round_by_round(first_round_matchups, n_sim=1000):
    round_results = {}
    round_details = {}
    current_matchups = first_round_matchups[:] 
    bar_length = 50  

    round_labels = {len(first_round_matchups): "Round of 64",
                    len(first_round_matchups)//2: "Round of 32",
                    len(first_round_matchups)//4: "Sweet 16",
                    len(first_round_matchups)//8: "Elite 8",
                    len(first_round_matchups)//16: "Final Four"}
    round_number = 0

    round_results["Round of 64"] = [f"{a} vs {b}" for a, b, _ in current_matchups]

    team_fatigue = defaultdict(lambda: 1.0)
    while len(current_matchups) > 1:
        round_number += 1
        label = round_labels.get(len(current_matchups), f"Round {round_number}")
        details, winners, next_matchups, team_fatigue = simulate_round_by_mode(current_matchups, n_sim, round_number, team_fatigue)
        round_results[label] = winners
        round_details[label] = details
        current_matchups = next_matchups
    if len(current_matchups) == 1:
        round_number += 1
        champ_details, champ_winners, _, team_fatigue = simulate_round_by_mode(current_matchups, n_sim, round_number, team_fatigue)
        round_results["Championship"] = champ_winners
        round_details["Championship"] = champ_details
        champion = champ_winners[0]
    else:
        champion = winners[0]
    round_results["Champion"] = champion
    round_details["Champion"] = champion
    print(f"Simulation Progress: [{'#' * bar_length}] 100.00% completed")
    return round_results, round_details

def print_round_details(round_name, details, n_sim=1000):

    print(f"\n{round_name}:")
    for team_a, team_b, best_team, count in details:
        percent = count / n_sim * 100
        print(f"{team_a} vs {team_b} -> {BOLD}{best_team}{RESET} ({count}/{n_sim} outcomes, {percent:.2f}%)")


def print_round_details(round_name, details, n_sim=1000):
    print(f"\n{round_name}:")
    for team_a, team_b, best_team, count, avg_p, closeness, fatigue in details:
        percent = count / n_sim * 100
        print(f"{team_a} vs {team_b} -> {BOLD}{best_team}{RESET} "
              f"({count}/{n_sim} outcomes, {percent:.2f}%)")



In [1236]:
#shared functions

def win_probability(team_a, team_b, team_fatigue_a=1.0,team_fatigue_b=1.0, df_degrees=3, round_number=1, verbose=False):

    # Base efficiency (means) from teams dictionary
    mu_a = teams[team_a] / team_fatigue_a
    mu_b = teams[team_b] / team_fatigue_b 

    # Draw heavy-tailed shocks for each team from a t-distribution we want to have something that mimics upsets
    shock_a = t.rvs(df_degrees) * params["shock_scale"]
    shock_b = t.rvs(df_degrees) * params["shock_scale"]
    
    # Get Luck values (default to 0 if not available)
    luck_a = team_luck[team_a] if 'team_luck' in globals() and team_a in team_luck else 0
    luck_b = team_luck[team_b] if 'team_luck' in globals() and team_b in team_luck else 0
    global_max_luck = np.abs(np.array(list(team_luck.values()))).max()
    luck_diff = (luck_a - luck_b)/global_max_luck
    
    # Get Momentum values (default to 0 if not available)
    momentum_a = team_momentum[team_a] if 'team_momentum' in globals() and team_a in team_momentum else 0
    momentum_b = team_momentum[team_b] if 'team_momentum' in globals() and team_b in team_momentum else 0
    global_max_tmmmt = np.abs(np.array(list(team_momentum.values()))).max()
    momentum_diff =  (momentum_a - momentum_b)/global_max_tmmmt

    # For the style adjustment, compute global mean and std of Adjusted Tempo ("AdjT")
    global_mean_adjt = tournament_df["AdjT"].mean()
    global_std_adjt = tournament_df["AdjT"].std()

    # Retrieve each team's Adjusted Tempo from the DataFrame.
    adjt_a = tournament_df.loc[tournament_df["Team"] == team_a, "AdjT"].iloc[0]
    adjt_b = tournament_df.loc[tournament_df["Team"] == team_b, "AdjT"].iloc[0]
    adjt_a /= global_mean_adjt
    adjt_b /= global_mean_adjt
    
    # Compute z-scores for the Adjusted Tempo values.
    adjt_a = (adjt_a - global_mean_adjt) / global_std_adjt
    adjt_b = (adjt_b - global_mean_adjt) / global_std_adjt
    
    style_diff = adjt_a - adjt_b


    effective_sigma = (team_sigma[team_a] + team_sigma[team_b])/ 2.  # Average both teams' sigma

    # Combine all effects into an effective difference:
    effective_diff =  (mu_a + shock_a) - (mu_b + shock_b)  \
                     + params["luck_factor"] * luck_diff \
                     + params["momentum_factor"] * momentum_diff \
                     + params["style_factor"] *style_diff

    p = 1 / (1 + np.exp(-effective_diff / effective_sigma))
    
    if(verbose):
        print(f"mua {mu_a}, shock_a {shock_a}")
        print(f"mub {mu_b}, shock_b {shock_b}")
        print(f"mu_eff {(mu_a + shock_a) - (mu_b + shock_b)}\n")
    
        print(f"luck {params['luck_factor'] * luck_diff}")
        print(f"momentum {params['momentum_factor'] * momentum_diff}")
        print(f"style {params['style_factor'] *style_diff}\n")
        print(f"effective_diff {effective_diff}, prob {p}")
    return p

def simulate_game(matchup, round_number=1, team_fatigue=None):

    if team_fatigue is None:
        team_fatigue = defaultdict(lambda: 1.0)
    if len(matchup) == 3:
        team_a, team_b, _ = matchup
    else:
        team_a, team_b = matchup
    
    p = win_probability(team_a, team_b, team_fatigue[team_a],team_fatigue[team_b])
    if np.random.rand() < p:
        winner = team_a
    else:
        winner = team_b
    return (winner,), p
    
def generate_first_round_matchups(main_bracket_teams):
    matchups = []
    for region in [ "West", "South", "East","Midwest"]:
        region_teams = main_bracket_teams[main_bracket_teams["Region"] == region].sort_values(by="Seed")
        first_round = [
            (region_teams.iloc[0]["Team"], region_teams.iloc[15]["Team"], region),
            (region_teams.iloc[7]["Team"], region_teams.iloc[8]["Team"], region),
            (region_teams.iloc[4]["Team"], region_teams.iloc[11]["Team"], region),
            (region_teams.iloc[3]["Team"], region_teams.iloc[12]["Team"], region),
            (region_teams.iloc[5]["Team"], region_teams.iloc[10]["Team"], region),
            (region_teams.iloc[2]["Team"], region_teams.iloc[13]["Team"], region),
            (region_teams.iloc[6]["Team"], region_teams.iloc[9]["Team"], region),
            (region_teams.iloc[1]["Team"], region_teams.iloc[14]["Team"], region),
        ]
        matchups.extend(first_round)
    return matchups

In [1237]:
nsim = 1000

params = { "shock_scale":0.05,
           "luck_factor":0.05,
           "momentum_factor":0.05,
           "style_factor":0.05,
           "fatigue_param":0.05
        }

In [ ]:
#Simulate statistics for most probable winner
full_bracket_paths = simulate_full_bracket(first_round_matchups, n_sim=nsim)


In [ ]:
#Print results
champion_counts = count_champions(full_bracket_paths)

print_champion_counts(champion_counts, n_sim=nsim)

In [ ]:
#simulate round by round to compute most probable winner of each game
round_results, round_details = simulate_tournament_round_by_round(first_round_matchups, n_sim=nsim)


In [ ]:
print("Tournament Outcome (mode-based round-by-round simulation):")
print("-----------------------------------------------------------")

for rnd in ["Round of 64", "Round of 32", "Sweet 16", "Elite 8", "Final Four", 'Championship']:
    if rnd in round_details:
        print_round_details(rnd, round_details[rnd], n_sim=nsim)

# Print the champion.
print(f"\nChampion: {BOLD}{round_details['Champion']}{RESET}")

